Cargar los datos

In [1]:
import os
import shutil
import yaml

import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from collections import defaultdict

C:\Users\Miguel Alejandro\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:

# Ruta al archivo dataset.yaml
yaml_path = "path/to/dataset.yaml"

# Clases que quieres conservar (según el índice en los archivos .txt de etiquetas)
clases_deseadas = {1, 3}  # Modifica según tus necesidades

# Cargar el archivo YAML
with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

# Obtener directorios desde el YAML
image_dirs = [data["train"], data.get("val", ""), data.get("test", "")]# cambiar
label_dirs = [d.replace("images", "labels") for d in image_dirs if d] # cambiar

def tiene_clase_deseada(label_file, clases_deseadas):
    """Verifica si el archivo de etiqueta contiene alguna de las clases deseadas."""
    with open(label_file, "r") as f:
        for line in f:
            class_id = int(line.split()[0])  # Primer número es la clase
            if class_id in clases_deseadas:
                return True
    return False

# Procesar cada conjunto (train, val, test si existen)
for image_dir, label_dir in zip(image_dirs, label_dirs):
    if not os.path.exists(label_dir):
        continue  # Saltar si no existe la carpeta de etiquetas

    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)
        image_path = os.path.join(image_dir, label_file.replace(".txt", ".jpg"))  # Ajusta si las imágenes son .png

        if not tiene_clase_deseada(label_path, clases_deseadas):
            os.remove(label_path)  # Eliminar etiqueta
            if os.path.exists(image_path):
                os.remove(image_path)  # Eliminar imagen

print("Filtrado completado.")


Crear conjuntos de prueba

sdadsfSSSSSSSSSSSSSSSSSSSSSSSSS

In [2]:
import json
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from collections import defaultdict

C:\Users\Miguel Alejandro\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Funciones útiles

In [12]:
def calcular_iou(box1, box2):
    """Calcula el Intersection over Union (IoU) entre dos cajas [x_min, y_min, x_max, y_max]."""
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    # Calcular el área de intersección
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)

    # Calcular el área de ambas cajas
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calcular el IoU
    iou = inter_area / float(box1_area + box2_area - inter_area) if (box1_area + box2_area - inter_area) > 0 else 0
    return iou

# def calcular_metricas_con_iou(predicciones, ground_truth, iou_threshold=0.5):
#     """Calcula precisión, recall y F1-score considerando IoU."""
#     resultado = {"precision": 0, "recall": 0, "f1_score": 0, "class_comparison": {}}

#     clases = set(predicciones.keys()) | set(ground_truth.keys())
#     total_tp, total_fp, total_fn = 0, 0, 0

#     for clase in clases:
#         pred_boxes = predicciones.get(clase, [])
#         gt_boxes = ground_truth.get(clase, [])
#         tp = 0
#         matched_gt = set()

#         for pred_box in pred_boxes:
#             mejor_iou = 0
#             mejor_gt_index = -1

#             # Comparar con cada caja de la verdad
#             for i, gt_box in enumerate(gt_boxes):
#                 iou = calcular_iou(pred_box, gt_box)
#                 if iou > mejor_iou and iou >= iou_threshold and i not in matched_gt:
#                     mejor_iou = iou
#                     mejor_gt_index = i

#             if mejor_gt_index != -1:
#                 tp += 1
#                 matched_gt.add(mejor_gt_index)

#         fp = len(pred_boxes) - tp  # Falsos Positivos: detecciones incorrectas
#         fn = len(gt_boxes) - tp  # Falsos Negativos: detecciones faltantes

#         total_tp += tp
#         total_fp += fp
#         total_fn += fn

#         resultado["class_comparison"][clase] = {"tp": tp, "fp": fp, "fn": fn}

#     # Cálculo de precisión, recall y F1-score
#     precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
#     recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
#     f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

#     resultado["precision"] = precision
#     resultado["recall"] = recall
#     resultado["f1_score"] = f1

#     return resultado

def contar_detecciones_por_clase(data, selected_classes):
    class_counts = defaultdict(int)
    
    for detections in data.values():  # Itera directamente sobre los valores (listas de detecciones)
        for detection in detections:
            class_name = detection['class']
            if class_name in selected_classes:  # Filtrar solo las clases de interés
                class_counts[class_name] += 1
    
    return dict(class_counts)

def calcular_metricas_con_iou(predicciones, ground_truth, selected_classes=None, iou_threshold=0.3):
    """Calcula precisión, recall y F1-score considerando IoU para las clases seleccionadas."""
    resultado = {"precision": 0, "recall": 0, "f1_score": 0}

    # Si no se especifican clases, usar todas las disponibles
    if selected_classes is None:
        selected_classes = set(predicciones.keys()) | set(ground_truth.keys())

    total_tp, total_fp, total_fn = 0, 0, 0

    for clase in selected_classes:
        pred_boxes = predicciones.get(clase, [])
        gt_boxes = ground_truth.get(clase, [])
        tp = 0
        matched_gt = set()

        for pred_box in pred_boxes:
            mejor_iou = 0
            mejor_gt_index = -1

            for i, gt_box in enumerate(gt_boxes):
                iou = calcular_iou(pred_box["bbox"], gt_box["bbox"])
                if iou > mejor_iou and iou >= iou_threshold and i not in matched_gt:
                    mejor_iou = iou
                    mejor_gt_index = i

            if mejor_gt_index != -1:
                tp += 1
                matched_gt.add(mejor_gt_index)

        fp = len(pred_boxes) - tp  # Falsos positivos
        fn = len(gt_boxes) - tp  # Falsos negativos

        total_tp += tp
        total_fp += fp
        total_fn += fn


    # Calcular precisión y recall
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0

    # Calcular F1-score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    resultado["precision"] = precision
    resultado["recall"] = recall
    resultado["f1_score"] = f1

    return resultado
# Crear DataFrames para mostrar en tabla
def generar_dataframe(data):
    df_data = []
    for modelo, metricas in data.items():
        for idx, metrica in enumerate(metricas):
            df_data.append({"modelo": modelo, "iteración": idx + 1, **metrica})
    return pd.DataFrame(df_data) 


def calcular_metricas(conteo_modelo, conteo_gt):
    """Calcula precisión y recall basados en los conteos de clases."""
    clases = set(conteo_modelo.keys()) | set(conteo_gt.keys())
    resultado = {"precision": 0, "recall": 0, "f1_score": {}}

    total_tp, total_fp, total_fn = 0, 0, 0

    for clase in clases:
        tp = min(conteo_modelo.get(clase, 0), conteo_gt.get(clase, 0))  # Verdaderos Positivos: lo que coincida
        fp = max(0, conteo_modelo.get(clase, 0) - tp)  # Falsos Positivos: detecciones extra
        fn = max(0, conteo_gt.get(clase, 0) - tp)  # Falsos Negativos: detecciones faltantes

        total_tp += tp
        total_fp += fp
        total_fn += fn

        # resultado["class_comparison"][clase] = {"tp": tp, "fp": fp, "fn": fn}
 
    # Calcular presición y recall
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    # Calcular F1-score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    resultado["precision"] = precision
    resultado["recall"] = recall
    # print("Precision: ", precision)
    # print("Recall: ", recall)
    resultado['f1_score'] = f1
    print("F1-score: ", f1)
    return resultado

In [4]:
# Cargar los archivos JSON
models = ["YOLO","CBR1","CBR2","CBR3","CBR4"]
paths = ["./Results/CBR1","./Results/CBR2","./Results/CBR3","./Results/CBR4","./Results/YOLO"]
results = ["/results1.json","/results2.json","/results3.json","/results4.json","/results5.json"]
path_gt = "./Results/Truth"

data = {}
for i in range(len(models)):
    data[models[i]] = []
    for elm in results:
        with open(paths[i]+elm) as f:
            model_data = json.load(f)
            data[models[i]].append(model_data)
ground_truth_data = []

for i in range(len(results)):
    with open(path_gt + results[i]) as f:
        ground_truth_data.append(json.load(f))


In [14]:
results  = {}
correcto = [] 
for i in range(len(ground_truth_data)):  
    conteo = contar_detecciones_por_clase(ground_truth_data[i], ["Cladosporium", "Curvularia"])
    # print(conteo)
    correcto.append(conteo)  
        
# Calcular métricas para cada modelo en "data"
for model, lista in data.items():  
    results[model] = []  

    for i in range(len(lista)): 
        metricas = calcular_metricas(contar_detecciones_por_clase(lista[i], ["Cladosporium", "Curvularia"]), correcto[i])
        results[model].append(metricas)


# # Calcular métricas para cada modelo en "data"
# for model, lista in data.items():  
#     results[model] = []  

#     for i in range(len(lista)):
        
#         metricas = calcular_metricas_con_iou(lista[i], ground_truth_data[i], ["Cladosporium", "Curvularia"])
#         print(metricas)
#         results[model].append(metricas)
        

F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0
F1-score:  1.0


In [15]:
df = generar_dataframe(results)
# Organizar datos en formato ancho
df_pivot = df.pivot(index='iteración', columns='modelo', values='f1_score').dropna()

# Convertir a formato largo
df_long = df_pivot.reset_index().melt(id_vars=['iteración'], var_name='modelo', value_name='f1_score')

# ANOVA de medidas repetidas
modelo_ols = ols('f1_score ~ C(modelo) + C(iteración)', data=df_long).fit()
anova_table = sm.stats.anova_lm(modelo_ols, typ=2)
print("\nTabla ANOVA:")
print(anova_table)

# Alternativamente, ANOVA con scipy.stats
f1_score_groups = [df_pivot[modelo].values for modelo in df_pivot.columns]
anova_result = stats.f_oneway(*f1_score_groups)
print("\nResultados del ANOVA (scipy.stats):")
print("F-statistic:", anova_result.statistic)
print("p-value:", anova_result.pvalue)


Tabla ANOVA:
                    sum_sq    df         F    PR(>F)
C(modelo)     4.901539e-31   4.0  0.263351  0.897161
C(iteración)  7.836005e-32   4.0  0.042101  0.996280
Residual      7.444875e-30  16.0       NaN       NaN

Resultados del ANOVA (scipy.stats):
F-statistic: nan
p-value: nan


c:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:3895: ConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(stats.ConstantInputWarning(msg))


In [16]:

# Comprobar supuestos del ANOVA: Normalidad y Homogeneidad de varianzas
# Prueba de normalidad (Shapiro-Wilk)
shapiro_test = stats.shapiro(df_long['f1_score'])
print("\nPrueba de normalidad (Shapiro-Wilk):", shapiro_test)

# Prueba de homogeneidad de varianzas (Levene)
levene_test = stats.levene(*[df_pivot[modelo].values for modelo in df_pivot.columns])
print("\nPrueba de homogeneidad de varianzas (Levene):", levene_test)



Prueba de normalidad (Shapiro-Wilk): ShapiroResult(statistic=1.0, pvalue=1.0)

Prueba de homogeneidad de varianzas (Levene): LeveneResult(statistic=nan, pvalue=nan)


c:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
c:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_morestats.py:2710: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom


In [17]:
# # Si ANOVA no es válido, usar Test de Friedman
# test_friedman = stats.friedmanchisquare(*[df_pivot[model].values for model in model_files])
# print("\nResultados del test de Friedman:")
# print("Chi-cuadrado:", test_friedman.statistic)
# print("p-value:", test_friedman.pvalue)

# Si ANOVA no es válido, usar Test de Friedman
test_friedman = stats.friedmanchisquare(*[df_pivot[modelo].values for modelo in df_pivot.columns])
print("\nResultados del Test de Friedman:")
print("Chi-cuadrado:", test_friedman.statistic)
print("p-value:", test_friedman.pvalue)



Resultados del Test de Friedman:
Chi-cuadrado: nan
p-value: nan


c:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:8696: RuntimeWarning: invalid value encountered in scalar divide
  chisq = (12.0 / (k*n*(k+1)) * ssbn - 3*n*(k+1)) / c


In [1]:
!pip show scikit-posthocs

Name: scikit-posthocs
Version: 0.11.2
Summary: Statistical post-hoc analysis and outlier detection algorithms
Home-page: 
Author: 
Author-email: Maksim Terpilovskii <maximtrp@gmail.com>
License: Copyright (c) 2024 Maksim Terpilovskii

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. I

In [18]:

import scikit_posthocs as sp
# Si el Test de Friedman es significativo, realizar la prueba post-hoc de Nemenyi
if test_friedman.pvalue < 0.05:
    print("\nPrueba post-hoc de Nemenyi:")
    nemenyi_results = sp.posthoc_nemenyi_friedman(df_pivot.values)
    print(nemenyi_results)
